In [12]:
from ipywidgets import interact
import os
from matplotlib import pyplot as plt
import ffmpeg
import numpy as np

from utils import get_img_frame, output_dir, get_resolution

In [20]:
input_video = os.path.abspath('11.webm')
input_png = os.path.abspath('grid.jpg')
file_name = os.path.splitext(os.path.basename(input_video))[0]
output_video = os.path.join(output_dir, 'test_mosh_{}.mp4'.format(
    file_name))  # this ensures we won't over-write your original video


probe = ffmpeg.probe(input_video)
video_info = next(s for s in probe['streams'] if s['codec_type'] == 'video')
width = int(video_info['width'])
height = int(video_info['height'])
num_frames = 378


overlay_ffmpeg = ffmpeg.input(input_png, s='{}x{}'.format(width, height))
fps = 25
start_sec = 0
end_sec = 20

In [19]:
video_info

{'avg_frame_rate': '4520/151',
 'chroma_location': 'left',
 'codec_long_name': 'On2 VP8',
 'codec_name': 'vp8',
 'codec_tag': '0x0000',
 'codec_tag_string': '[0][0][0][0]',
 'codec_time_base': '151/4520',
 'codec_type': 'video',
 'coded_height': 640,
 'coded_width': 640,
 'display_aspect_ratio': '1:1',
 'disposition': {'attached_pic': 0,
  'clean_effects': 0,
  'comment': 0,
  'default': 1,
  'dub': 0,
  'forced': 0,
  'hearing_impaired': 0,
  'karaoke': 0,
  'lyrics': 0,
  'original': 0,
  'timed_thumbnails': 0,
  'visual_impaired': 0},
 'field_order': 'progressive',
 'has_b_frames': 0,
 'height': 640,
 'index': 0,
 'level': -99,
 'pix_fmt': 'yuv420p',
 'profile': '0',
 'r_frame_rate': '4520/151',
 'refs': 1,
 'sample_aspect_ratio': '1:1',
 'start_pts': 0,
 'start_time': '0.000000',
 'tags': {'DURATION': '00:00:15.100000000', 'ENCODER': 'Lavc58.11.101 libvpx'},
 'time_base': '1/1000',
 'width': 640}

In [55]:
out1, err1 = (
    ffmpeg
        .input(input_png, s='{}x{}'.format(width, height))
        .output('pipe:', format='rawvideo', pix_fmt='yuv420p',s='{}x{}'.format(width, height))
        .run(capture_stdout=True)
)

out1

b'\xeb\xeb\xec\xaf\r\x10\x10\x10\x07c\xec\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xec\xe1-\x0b\x10\x10\x0c(\xde\xec\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xecj\x07\x10\x10\x10\n\xa9\xec\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xec\xb0\x0c\x0f\x10\x10\x07c\xec\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xec\xe2-\x0b\x10\x10\x0c(\xde\xec\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xec\xe2-\x0b\x10\x10\x0c(\xde\xec\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xecj\x07\x10\x10\x10\n\xa9\xec\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xec\xb0\x0c\x0f\x10\x10\x07c\xec\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xec\xe1-\x0b\x10\x10\x0c(\xde\xec\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xecj\x07\x10\x10\x10\x0b\xa9\xec\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xec\xb0\r\x0f\x10\x10\x07c\xec\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xeb\xe

In [48]:
out, err = (
    ffmpeg
    .input('moshed_videos/temp_mosh_11.avi')
    .overlay(overlay_ffmpeg, eof_action='pass')
    .output('pipe:', format='yuv420', pix_fmt='rgb24')
    .run(capture_stdout=True)
)
video = (
    np
    .frombuffer(out, np.uint8)
    .reshape([-1, height, width, 3])
)

@interact(frame=(0, num_frames))
def show_frame(frame=0):
    plt.imshow(video[frame,:,:,:])



Error: ffmpeg error (see stderr output for detail)